二值图像：OpenCV中二值图像为单通道的、字节类型的Mat对象、对于任意的输入图像首先需要把图像转换为灰度、然后通过二值化方法转换为二值图像。
本文首先介绍二值化的方法，包括阈值分割函数的使用、otsu法、triangle法、自适应局部阈值法以及二值化方法和去噪方法的结合。
然后介绍连通域的获取和分析；轮廓的获取和分析。
最后介绍一下霍夫变换的应用。

取图像均值后二值化

In [2]:
import cv2 as cv
import numpy as np

src = cv.imread("C:/Users/86188/Desktop/test/other/shihuang.png")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)

T = 127

# 转换为灰度图像
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
h, w = gray.shape
print(cv.mean(gray))
T = cv.mean(gray)[0]
print("current threshold value : ", T)

# 二值图像
binary = np.zeros((h, w), dtype=np.uint8)
for row in range(h):
    for col in range(w):
        pv = gray[row, col]
        if pv > T:
            binary[row, col] = 255
        else:
            binary[row, col] = 0
cv.imshow("binary", binary)

cv.waitKey(0)
cv.destroyAllWindows()

(194.1143267598276, 0.0, 0.0, 0.0)
current threshold value :  194.1143267598276


阈值分割函数  double cv::threshold(InputArray src, OutputArray dst,double thresh,double maxval,int type)

In [3]:
import cv2 as cv
import numpy as np
#
# THRESH_BINARY = 0
# THRESH_BINARY_INV = 1
# THRESH_TRUNC = 2
# THRESH_TOZERO = 3
# THRESH_TOZERO_INV = 4
#
src = cv.imread("C:/Users/86188/Desktop/test/other/shihuang.png")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)

T = 127

# 转换为灰度图像
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
for i in range(5):
    ret, binary = cv.threshold(gray, T, 255, i)
    cv.imshow("binary_" + str(i), binary)

cv.waitKey(0)
cv.destroyAllWindows()

上面两个例子阈值分别是取平均和手动设置，下面介绍寻找阈值的方法

OTSU（大津法）：通过计算类间最大方差来确定分割阈值的阈值选择算法，使用于双峰图像

In [13]:
import cv2 as cv
import numpy as np
#
# THRESH_BINARY = 0
# THRESH_BINARY_INV = 1
# THRESH_TRUNC = 2
# THRESH_TOZERO = 3
# THRESH_TOZERO_INV = 4
#
src = cv.imread("C:/Users/86188/Desktop/test/other/shihuang.png")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)
h, w = src.shape[:2]

# 自动阈值分割 OTSU
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
print("ret :", ret)
cv.imshow("binary", binary)

result = np.zeros([h, w*2, 3], dtype=src.dtype)
result[0:h,0:w,:] = src
result[0:h,w:2*w,:] = cv.cvtColor(binary, cv.COLOR_GRAY2BGR)
cv.putText(result, "input", (10, 30), cv.FONT_ITALIC, 1.0, (0, 0, 255), 2)
cv.putText(result, "binary, threshold = " + str(ret), (w+10, 30), cv.FONT_ITALIC, 1.0, (0, 0, 255), 2)
cv.imshow("result", result)

cv.waitKey(0)
cv.destroyAllWindows()

ret : 147.0


In [ ]:
Triangle（三角）阈值分割：适用于单峰情况

In [12]:
import cv2 as cv
import numpy as np
import tensorflow as tf

#
# THRESH_BINARY = 0
# THRESH_BINARY_INV = 1
# THRESH_TRUNC = 2
# THRESH_TOZERO = 3
# THRESH_TOZERO_INV = 4
#
src = cv.imread("C:/Users/86188/Desktop/test/other/shihuang.png")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)
h, w = src.shape[:2]


# 自动阈值分割 TRIANGLE
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_TRIANGLE)
print("ret :", ret)
cv.imshow("binary", binary)

result = np.zeros([h, w*2, 3], dtype=src.dtype)
result[0:h,0:w,:] = src
result[0:h,w:2*w,:] = cv.cvtColor(binary, cv.COLOR_GRAY2BGR)
cv.putText(result, "input", (10, 30), cv.FONT_ITALIC, 0.5, (0, 0, 255), 1)
cv.putText(result, "binary,threshold = " + str(ret), (w+10, 30), cv.FONT_ITALIC, 0.5, (0, 0, 255), 1)
cv.imshow("result", result)

cv.waitKey(0)
cv.destroyAllWindows()

ret : 164.0


In [ ]:
Adaptivethreshold自适应阈值：在同一幅图像的不同部分具有不同亮度时使用，局部阈值（每个点的阈值受到邻近的block影响）

In [14]:
import cv2 as cv
import numpy as np

#
# THRESH_BINARY = 0
# THRESH_BINARY_INV = 1
# THRESH_TRUNC = 2
# THRESH_TOZERO = 3
# THRESH_TOZERO_INV = 4
#
src = cv.imread("C:/Users/86188/Desktop/test/other/text1.png")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)
h, w = src.shape[:2]


# 自动阈值分割 TRIANGLE
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
binary = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 25, 10)
cv.imshow("binary", binary)

result = np.zeros([h, w*2, 3], dtype=src.dtype)
result[0:h,0:w,:] = src
result[0:h,w:2*w,:] = cv.cvtColor(binary, cv.COLOR_GRAY2BGR)
cv.putText(result, "input", (10, 30), cv.FONT_ITALIC, 1.0, (0, 0, 255), 2)
cv.putText(result, "adaptive threshold", (w+10, 30), cv.FONT_ITALIC, 1.0, (0, 0, 255), 2)
cv.imshow("result", result)

cv.waitKey(0)
cv.destroyAllWindows()

图像去噪后二值化

In [1]:
import cv2 as cv
import numpy as np


def method_1(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    t, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    return binary


def method_2(image):
    blurred = cv.GaussianBlur(image, (3, 3), 0)
    gray = cv.cvtColor(blurred, cv.COLOR_BGR2GRAY)
    t, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    return binary


def method_3(image):
    blurred = cv.pyrMeanShiftFiltering(image, 10, 100)
    gray = cv.cvtColor(blurred, cv.COLOR_BGR2GRAY)
    t, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    return binary


src = cv.imread("C:/Users/86188/Desktop/test/other/shihuang.png")
h, w = src.shape[:2]
ret = method_3(src)

result = np.zeros([h, w*2, 3], dtype=src.dtype)
result[0:h,0:w,:] = src
result[0:h,w:2*w,:] = cv.cvtColor(ret, cv.COLOR_GRAY2BGR)
cv.putText(result, "input", (10, 30), cv.FONT_ITALIC, 1.0, (0, 0, 255), 2)
cv.putText(result, "binary", (w+10, 30), cv.FONT_ITALIC, 1.0, (0, 0, 255), 2)
cv.imshow("result", result)

cv.waitKey(0)
cv.destroyAllWindows()

二值图像的各类操作，接下来首先介绍二值图像下的连通域操作

联通组件标记算法

In [4]:
import cv2 as cv
import numpy as np


def connected_components_demo(src):
    src = cv.GaussianBlur(src, (3, 3), 0)
    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    cv.imshow("binary", binary)

    output = cv.connectedComponents(binary, connectivity=8, ltype=cv.CV_32S)
    num_labels = output[0]
    labels = output[1]
    colors = []
    for i in range(num_labels):
        b = np.random.randint(0, 256)
        g = np.random.randint(0, 256)
        r = np.random.randint(0, 256)
        colors.append((b, g, r))

    colors[0] = (0, 0, 0)
    h, w = gray.shape
    image = np.zeros((h, w, 3), dtype=np.uint8)
    for row in range(h):
        for col in range(w):
            image[row, col] = colors[labels[row, col]]

    cv.imshow("colored labels", image)
    print("total rice : ", num_labels - 1)


src = cv.imread("C:/Users/86188/Desktop/test/other/rice.png")
connected_components_demo(src)
cv.waitKey(0)
cv.destroyAllWindows()

total rice :  25


每个区域的统计特征，外接矩形的左上角坐标和长宽，连通域面积，连通域中心

In [6]:
import cv2 as cv
import numpy as np


def connected_components_stats_demo(src):
    src = cv.GaussianBlur(src, (3, 3), 0)
    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    cv.imshow("binary", binary)

    num_labels, labels, stats, centers = cv.connectedComponentsWithStats(binary, connectivity=8, ltype=cv.CV_32S)
    colors = []
    for i in range(num_labels):
        b = np.random.randint(0, 256)
        g = np.random.randint(0, 256)
        r = np.random.randint(0, 256)
        colors.append((b, g, r))

    colors[0] = (0, 0, 0)
    image = np.copy(src)
    for t in range(1, num_labels, 1):
        x, y, w, h, area = stats[t]
        cx, cy = centers[t]
        cv.circle(image, (np.int32(cx), np.int32(cy)), 2, (0, 255, 0), 2, 8, 0)
        cv.rectangle(image, (x, y), (x+w, y+h), colors[t], 1, 8, 0)
        cv.putText(image, "num:" + str(t), (x, y), cv.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1);
        print("label index %d, area of the label : %d"%(t, area))

    cv.imshow("colored labels", image)
    print("total rice : ", num_labels - 1)


input = cv.imread("C:/Users/86188/Desktop/test/other/rice.png")
connected_components_stats_demo(input)
cv.waitKey(0)
cv.destroyAllWindows()

label index 1, area of the label : 2022
label index 2, area of the label : 1955
label index 3, area of the label : 1900
label index 4, area of the label : 1844
label index 5, area of the label : 1861
label index 6, area of the label : 1885
label index 7, area of the label : 1831
label index 8, area of the label : 1916
label index 9, area of the label : 2017
label index 10, area of the label : 2041
label index 11, area of the label : 1970
label index 12, area of the label : 1966
label index 13, area of the label : 1983
label index 14, area of the label : 1841
label index 15, area of the label : 2068
label index 16, area of the label : 1903
label index 17, area of the label : 1889
label index 18, area of the label : 1509
label index 19, area of the label : 1770
label index 20, area of the label : 1566
label index 21, area of the label : 2018
label index 22, area of the label : 1962
label index 23, area of the label : 2011
label index 24, area of the label : 1812
label index 25, area of t

轮廓外接矩形
cv.findContours
cv.drawContours

In [16]:
import cv2 as cv
import numpy as np


def canny_demo(image):
    t = 80
    canny_output = cv.Canny(image, t, t * 2)
    cv.imshow("canny_output", canny_output)
    return canny_output


src = cv.imread("C:/Users/86188/Desktop/test/other/stuff.jpg")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)
binary = canny_demo(src)
cv.imshow("canny1",binary)
k = np.ones((3, 3), dtype=np.uint8)
binary = cv.morphologyEx(binary, cv.MORPH_DILATE, k)
cv.imshow("canny2",binary)
# 轮廓发现
out, contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
for c in range(len(contours)):
    #轮廓面积和弧长
    area = cv.contourArea(contours[c])
    arclen = cv.arcLength(contours[c], True)
    if area>1200:
        continue
    print(area,arclen)
     #直外接矩形的参数
    x, y, w, h = cv.boundingRect(contours[c]);
    #绘制第c个轮廓
    cv.drawContours(src, contours, c, (0, 0, 255), 2, 8)
    #绘制直外接矩形
    cv.rectangle(src, (x, y), (x+w, y+h), (0, 0, 255), 1, 8, 0);
    #最小外接矩形，rect包括最小矩形的中心，长宽和旋转角度
    rect = cv.minAreaRect(contours[c])
    cx, cy = rect[0]
    #使用boxPoints函数计算出最小外接矩形的四个点坐标
    box = cv.boxPoints(rect)
    #将坐标化为整数
    box = np.int0(box)
    cv.drawContours(src,[box],0,(0,0,255),2)
    #绘制中心点
    cv.circle(src, (np.int32(cx), np.int32(cy)), 2, (255, 0, 0), 2, 8, 0)


# 显示
cv.imshow("contours_analysis", src)
cv.waitKey(0)
cv.destroyAllWindows()

1100.5 762.9188274145126
997.0 736.1564152240753


轮廓面积，轮廓周长，轮廓近似，轮廓几何矩
area = cv.contourArea(cnt)

轮廓周长
arclen = cv.arcLength(cnt, True)

轮廓近似
approx = cv.approxPolyDP(cnt,epsilon,True)

轮廓几何矩
mm = cv.moments(cnt)

最小外接圆
(x,y),radius = cv.minEnclosingCircle(cnt)
center = (int(x),int(y))
radius = int(radius)
img = cv.circle(img,center,radius,(0,255,0),2)

椭圆拟合
ellipse = cv.fitEllipse(cnt)

直线拟合
[vx,vy,x,y]=cv.fitLine(cnt,cv.DIST_L2,0,0.01,0.01)

凸包检测
cv.isContourConvex(cnt)
cv.convexHull(cnt)

点到轮廓的距离测试（点多边形测试）
dist = cv.pointPolygonTest(cnt, (col, row), True)

最大内接圆
opencv不直接提供最大内接园的API，可以通过点多边形测试获得距离轮廓最远的点，这个点与轮廓的距离就是最大内接圆的半径

使用Hu矩实现轮廓匹配：cv.HuMoments，cv.matchShapes

点到轮廓的距离测试（点多边形测试）

In [20]:
import cv2 as cv
import numpy as np

src = cv.imread("C:/Users/86188/Desktop/test/other/abc.png")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
cv.imshow("binary", binary)

# 轮廓发现
image = np.zeros(src.shape, dtype=np.float32)
out, contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
h, w = src.shape[:2]
for row in range(h):
    for col in range(w):
        dist = cv.pointPolygonTest(contours[0], (col, row), True)
        if dist == 0:
            image[row, col] = (255, 255, 255)
        if dist > 0:
            image[row, col] = (255-dist, 0, 0)
        if dist < 0:
            image[row, col] = (0, 0, 255+dist)

dst = cv.convertScaleAbs(image)
dst = np.uint8(dst)

# 显示
cv.imshow("contours_analysis", dst)
cv.waitKey(0)
cv.destroyAllWindows()

最大内接圆

In [23]:
from __future__ import print_function
from __future__ import division
import cv2 as cv
import numpy as np
# Create an image
r = 100
src = np.zeros((4*r, 4*r), dtype=np.uint8)
# Create a sequence of points to make a contour
vert = [None]*6
vert[0] = (3*r//2, int(1.34*r))
vert[1] = (1*r, 2*r)
vert[2] = (3*r//2, int(2.866*r))
vert[3] = (5*r//2, int(2.866*r))
vert[4] = (3*r, 2*r)
vert[5] = (5*r//2, int(1.34*r))
# Draw it in src
for i in range(6):
    cv.line(src, vert[i],  vert[(i+1)%6], ( 255 ), 3)
# Get the contours
_, contours, _ = cv.findContours(src, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# Calculate the distances to the contour
raw_dist = np.empty(src.shape, dtype=np.float32)
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        raw_dist[i,j] = cv.pointPolygonTest(contours[0], (j,i), True)

# 获取最大值即内接圆半径，中心点坐标
minVal, maxVal, _, maxDistPt = cv.minMaxLoc(raw_dist)
minVal = abs(minVal)
maxVal = abs(maxVal)

# Depicting the  distances graphically
drawing = np.zeros((src.shape[0], src.shape[1], 3), dtype=np.uint8)
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        if raw_dist[i,j] < 0:
            drawing[i,j,0] = 255 - abs(raw_dist[i,j]) * 255 / minVal
        elif raw_dist[i,j] > 0:
            drawing[i,j,2] = 255 - raw_dist[i,j] * 255 / maxVal
        else:
            drawing[i,j,0] = 255
            drawing[i,j,1] = 255
            drawing[i,j,2] = 255

# max inner circle
cv.circle(drawing,maxDistPt, np.int(maxVal),(255,255,255), 1, cv.LINE_8, 0)
cv.imshow('Source', src)
cv.imshow('Distance and inscribed circle', drawing)

cv.waitKey(0)
cv.destroyAllWindows()

中心距：平移不变性  归一化中心矩：尺度不变性  Hu矩：旋转不变性
使用Hu矩实现轮廓匹配：cv.HuMoments，cv.matchShapes

In [ ]:
import cv2 as cv
import numpy as np


def contours_info(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    out, contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    return contours


src = cv.imread("C:/Users/86188/Desktop/test/other/abc.png")
cv.namedWindow("input1", cv.WINDOW_AUTOSIZE)
cv.imshow("input1", src)
src2 = cv.imread("C:/Users/86188/Desktop/test/other/a5.png")
cv.imshow("input2", src2)

# 轮廓发现
contours1 = contours_info(src)
contours2 = contours_info(src2)

# 几何矩计算与hu矩计算
mm2 = cv.moments(contours2[0])
hum2 = cv.HuMoments(mm2)

# 轮廓匹配
for c in range(len(contours1)):
    mm = cv.moments(contours1[c])
    hum = cv.HuMoments(mm)
    dist = cv.matchShapes(hum, hum2, cv.CONTOURS_MATCH_I1, 0)
    if dist < 1:
        cv.drawContours(src, contours1, c, (0, 0, 255), 2, 8)
    print("dist %f"%(dist))

# 显示
cv.imshow("contours_analysis", src)
cv.waitKey(0)
cv.destroyAllWindows()

最后介绍一下霍夫变换。

In [10]:
import cv2 as cv
import numpy as np


def canny_demo(image):
    t = 80
    canny_output = cv.Canny(image, t, t * 2)
    return canny_output


src = cv.imread("C:/Users/86188/Desktop/test/other/sudoku.png")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)

binary = canny_demo(src)
cv.imshow("binary", binary)

lines = cv.HoughLines(binary, 1, np.pi / 180, 150, None, 0, 0)
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
        pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
        cv.line(src, pt1, pt2, (0, 0, 255), 3, cv.LINE_AA)

cv.imshow("hough line demo", src)
cv.waitKey(0)
cv.destroyAllWindows()

cv.HoughLinesP直接返回霍夫变换检测到的点

In [12]:
import cv2 as cv
import numpy as np


src = cv.imread("C:/Users/86188/Desktop/test/other/coins.jpg")
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)
cv.imshow("input", src)
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
gray = cv.GaussianBlur(gray, (9, 9), 2, 2)
dp = 2
param1 = 100
param2 = 80


circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, dp, 10, None, param1, param2, 20, 100)
for c in circles[0,:]:
    print(c)
    cx, cy, r = c
    cv.circle(src, (cx, cy), 2, (0, 255, 0), 2, 8, 0)
    cv.circle(src, (cx, cy), r, (0, 0, 255), 2, 8, 0)


cv.imshow("hough line demo", src)
cv.waitKey(0)
cv.destroyAllWindows()

[217.  105.   41.6]
[305. 121.  30.]
[257.  235.   39.4]
[327.  189.   39.2]
[193.  181.   41.6]
[323. 257.  30.]
[141. 115.  37.]
[187.  253.   30.4]
[261.  161.   30.6]
[191.  149.   54.8]
[207.  163.   48.2]
[233.  213.   61.4]
[209.  113.   41.6]
[333. 181.  50.]
[243.  231.   52.6]
[263.  227.   48.2]
[251. 165.  35.]
[149. 109.  46.]
[217. 159.  35.]
[321.  179.   50.4]
[209.   99.   32.8]
[227.  101.   30.6]
[229. 177.  35.]
